In [0]:
from google.colab import drive 
drive.mount('/content/drive')

In [0]:
#run
import os
import re
import gc
import json
import glob
import math
import time
import torch
import os,sys
import random
import string
import pickle
import logging
import itertools
import unicodedata
import torch.nn as nn
from fastai.imports import *
import torch.nn.functional as F
from torch import optim
from collections import Counter
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score
from __future__ import print_function, division, unicode_literals
from numpy.random import choice as random_choice,randint as random_randint, shuffle as random_shuffle, seed as random_seed, rand
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [0]:
#run
# Set a logger for the module
LOGGER = logging.getLogger(__name__) # Every log will use the module name
LOGGER.addHandler(logging.StreamHandler())
LOGGER.setLevel(logging.DEBUG)
random_seed(123)

In [0]:
#run
class Configuration(object):
    """Dump stuff here"""

CONFIG = Configuration()
#pylint:disable=attribute-defined-outside-init
# Parameters for the model:
CONFIG.input_layers = 2
CONFIG.output_layers = 2
CONFIG.amount_of_dropout = 0.2
CONFIG.hidden_size = 500
CONFIG.number_of_chars = 100
CONFIG.max_input_len = 60
CONFIG.inverted = False

# Parameters for the dataset
MIN_INPUT_LEN = 5
AMOUNT_OF_NOISE = 0.2 / CONFIG.max_input_len
CHARS = list("abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ .")
PADDING = "☕"

In [0]:
# RUN ONCE
#file_list = glob.glob("/content/drive/My Drive/1-billion/training/*.txt")

In [0]:
#os.listdir("/content/drive/My Drive/1-billion/heldout")

Run only once
> 
Reads all the txt files in variable file_list to result.txt

In [0]:
'''with open("/content/drive/My Drive/1-billion/result.txt", "wb") as outfile:
    for f in file_list:
        with open(f, "rb") as infile:
            outfile.write(infile.read())'''

'with open("/content/drive/My Drive/1-billion/result.txt", "wb") as outfile: #ONLY ONCE\n    for f in file_list:\n        with open(f, "rb") as infile:\n            outfile.write(infile.read())'

In [0]:
#run
DATA_FILES_FULL_PATH = '/content/drive/My Drive/1-billion/'
#NEWS_FILE_NAME = 'del_v2.txt'
NEWS_FILE_NAME = '/content/drive/My Drive/1-billion/result.txt'
NEWS_FILE_NAME_CLEAN = os.path.join(DATA_FILES_FULL_PATH, "news.2013.en.clean")
NEWS_FILE_NAME_FILTERED = os.path.join(DATA_FILES_FULL_PATH, "news.2013.en.filtered")
NEWS_FILE_NAME_SPLIT = os.path.join(DATA_FILES_FULL_PATH, "news.2013.en.split")
NEWS_FILE_NAME_TRAIN = os.path.join(DATA_FILES_FULL_PATH, "news.2013.en.train")
NEWS_FILE_NAME_VALIDATE = os.path.join(DATA_FILES_FULL_PATH, "news.2013.en.validate")
NEWS_FILE_NAME_TEST = os.path.join(DATA_FILES_FULL_PATH, "news.2013.en.test")

NEWS_FILE_NAME_TRAIN_20 = os.path.join(DATA_FILES_FULL_PATH, "news.2013.en.train_20")
NEWS_FILE_NAME_TRAIN_40 = os.path.join(DATA_FILES_FULL_PATH, "news.2013.en.train_40")
NEWS_FILE_NAME_TRAIN_60 = os.path.join(DATA_FILES_FULL_PATH, "news.2013.en.train_60")
NEWS_FILE_NAME_TRAIN_80 = os.path.join(DATA_FILES_FULL_PATH, "news.2013.en.train_80")
NEWS_FILE_NAME_TRAIN_90 = os.path.join(DATA_FILES_FULL_PATH, "news.2013.en.train_90")
NEWS_FILE_NAME_TRAIN_100 = os.path.join(DATA_FILES_FULL_PATH, "news.2013.en.train_100")

#NEWS_FILE_NAME_VALIDATE_20 = os.path.join(DATA_FILES_FULL_PATH, "news.2013.en.validate_20")
CHAR_FREQUENCY_FILE_NAME = os.path.join(DATA_FILES_FULL_PATH, "char_frequency.json")
SAVED_MODEL_FILE_NAME = os.path.join(DATA_FILES_FULL_PATH, "keras_spell_e{}.h5") # an HDF5 file

In [0]:
all_letters = all_letters = string.ascii_letters + " .,);'(-"

def unicodeToAscii(s):
    return ''.join(
        c for c in unicodedata.normalize('NFD', s)
        if unicodedata.category(c) != 'Mn'
        and c in all_letters
    )
def normalizeString(s):
    s = unicodeToAscii(s.strip())
    s = re.sub(r"([.!?])", r" \1", s)
    s = re.sub(r"[^a-zA-Z.!?]+", r" ", s)
    s = re.sub(r"\s+", r" ", s).strip()
    return s

def cleaned_text(text):
    result = re.compile(r'[^\S\n]+', re.UNICODE).sub(' ', text.strip())
    
    result = re.compile(r'[\-\˗\֊\‐\‑\‒\–\—\⁻\₋\−\﹣\－]', re.UNICODE).sub('-', result)
    
    result = re.compile(r'&#39;|[ʼ՚＇‘’‛❛❜ߴߵ`‵´ˊˋ{}{}{}{}{}{}{}{}{}]'.format(chr(768), chr(769), chr(832), chr(833), chr(2387), chr(5151),
                                                                                      chr(5152), chr(65344), chr(8242)), re.UNICODE).sub("'", result)
    result = re.compile(r'[\(\[\{\⁽\₍\❨\❪\﹙\（]', re.UNICODE).sub("(", result)
    
    result = re.compile(r'[\)\]\}\⁾\₎\❩\❫\﹚\）]', re.UNICODE).sub(")", result)
    
    result = re.compile(r'[^\w\s{}{}]'.format(re.escape("""¥£₪$€฿₨"""), re.escape("""-!?/;"'%&<>.()[]{}@#:,|=*""")), re.UNICODE).sub('', result)

    result = unicodeToAscii(result)
    result = normalizeString(result)

    return result


def preprocesses_data_clean():
    with open(NEWS_FILE_NAME_CLEAN, "w") as clean_data:
        for line in open(NEWS_FILE_NAME, 'r', encoding="utf-8"):
            cleaned_line = cleaned_text(line)
            clean_data.write(cleaned_line + "\n")

'from __future__ import print_function, division, unicode_literals            # RUN ONCE\npreprocesses_data_clean()'

In [0]:
# RUN ONCE
def preprocesses_data_analyze_chars():
    """Pre-process the data - step 2 - analyze the characters"""
    counter = Counter()
    LOGGER.info("Reading data:")
    for line in open(NEWS_FILE_NAME_CLEAN,  'r', encoding="utf-8"):
        decoded_line = line#.decode('utf-8')
        counter.update(decoded_line)
    LOGGER.info("Done.\nWriting to file:")
    with open(CHAR_FREQUENCY_FILE_NAME, 'w', encoding="utf-8") as output_file:
        output_file.write(json.dumps(counter))
    most_popular_chars = {key for key, _value in counter.most_common(CONFIG.number_of_chars)}
    LOGGER.info("The top %s chars are:", CONFIG.number_of_chars)
    LOGGER.info("".join(sorted(most_popular_chars)))


In [0]:
# RUN ONCE
def read_top_chars():
    """Read the top chars we saved to file"""
    chars = json.loads(open(CHAR_FREQUENCY_FILE_NAME).read())
    counter = Counter(chars)
    most_popular_chars = {key for key, _value in counter.most_common(CONFIG.number_of_chars)}
    return most_popular_chars

def preprocesses_data_filter():
    """Pre-process the data - step 3 - filter only sentences with the right chars"""
    most_popular_chars = read_top_chars()
    LOGGER.info("Reading and filtering data:")
    with open(NEWS_FILE_NAME_FILTERED, "w") as output_file:
        for line in open(NEWS_FILE_NAME_CLEAN, 'r', encoding='utf-8'):
            decoded_line = line#.decode('utf-8')
            if decoded_line and not bool(set(decoded_line) - most_popular_chars):
                output_file.write(line)
    LOGGER.info("Done.")


In [0]:
# RUN ONCE
def read_filtered_data():
    """Read the filtered data corpus"""
    LOGGER.info("Reading filtered data:")
    lines = open(NEWS_FILE_NAME_FILTERED).read().decode('utf-8').split("\n")
    LOGGER.info("Read filtered data - %s lines", len(lines))
    return lines

def preprocesses_split_lines():
    LOGGER.info("Reading filtered data:")
    answers = set()
    with open(NEWS_FILE_NAME_SPLIT, "wb") as output_file:
        for line in open(NEWS_FILE_NAME_FILTERED,'r'):
            #line = _line.decode('utf-8')
            while len(line) > MIN_INPUT_LEN:
                if len(line) <= CONFIG.max_input_len:
                    answer = line
                    line = ""
                else:
                    space_location = line.rfind(" ", MIN_INPUT_LEN, CONFIG.max_input_len - 1)
                    if space_location > -1:
                        answer = line[:space_location]
                        line = line[len(answer) + 1:]
                    else:
                        space_location = line.rfind(" ") # no limits this time
                        if space_location == -1:
                            break # we are done with this line
                        else:
                            line = line[space_location + 1:]
                            continue
                answers.add(answer)
                output_file.write(answer.encode('utf-8') + b"\n")



Processing the whole data was consuming up almost all the available RAM on colab (~25 GB). So i split the data into multiple parts and processesed the separately.

In [0]:
# RUN ONCE
def preprocess_partition_data():
    """Set asside data for validation"""
    answers = open(NEWS_FILE_NAME_SPLIT).read().split("\n")
    print('shuffle', end=" ")
    random_shuffle(answers)

    split_at = len(answers) // 5

    with open(NEWS_FILE_NAME_TRAIN_20, "wb") as output_file:
        output_file.write("\n".join(answers[:split_at]).encode('utf-8'))

    with open(NEWS_FILE_NAME_TRAIN_40, "wb") as output_file:
        output_file.write("\n".join(answers[split_at:2*split_at]).encode('utf-8'))

    with open(NEWS_FILE_NAME_TRAIN_60, "wb") as output_file:
        output_file.write("\n".join(answers[2*split_at:3*split_at]).encode('utf-8'))

    with open(NEWS_FILE_NAME_TRAIN_80, "wb") as output_file:
        output_file.write("\n".join(answers[3*split_at:4*split_at]).encode('utf-8'))

    with open(NEWS_FILE_NAME_TRAIN_90, "wb") as output_file:
        output_file.write("\n".join(answers[4*split_at:math.floor(4.5*split_at)]).encode('utf-8'))

    with open(NEWS_FILE_NAME_VALIDATE, "wb") as output_file:
        output_file.write("\n".join(answers[math.ceil(4.5*split_at):]).encode('utf-8'))



In [0]:
#run
def add_noise_to_string(a_string, amount_of_noise):
    """Add some artificial spelling mistakes to the string"""
    if rand() < amount_of_noise * len(a_string):
        # Replace a character with a random character
        random_char_position = random_randint(len(a_string))
        a_string = a_string[:random_char_position] + random_choice(CHARS[:-1]) + a_string[random_char_position + 1:]
    if rand() < amount_of_noise * len(a_string):
        # Delete a character
        random_char_position = random_randint(len(a_string))
        a_string = a_string[:random_char_position] + a_string[random_char_position + 1:]
    if len(a_string) < CONFIG.max_input_len and rand() < amount_of_noise * len(a_string):
        # Add a random character
        random_char_position = random_randint(len(a_string))
        a_string = a_string[:random_char_position] + random_choice(CHARS[:-1]) + a_string[random_char_position:]
    if rand() < amount_of_noise * len(a_string):
        # Transpose 2 characters
        random_char_position = random_randint(len(a_string) - 1)
        a_string = (a_string[:random_char_position] + a_string[random_char_position + 1] + a_string[random_char_position] +
                    a_string[random_char_position + 2:])
    return a_string


def generate_question(answer):
    """Generate a question by adding noise"""
    question = add_noise_to_string(answer, AMOUNT_OF_NOISE)
    return question, answer


def generate_news_data():
    """Generate some news data"""
    print ("Generating Data")
    with open(NEWS_FILE_NAME_VALIDATE,'r') as f:
      answers = f.read().split("\n")
    print('shuffle', end=" ")
    random_shuffle(answers)
    print("Done")
    pairs = []
    lolz = len(answers)
    nn=0

    start = time.time()
 
    for answer in answers:
      if answer != "":
        question = add_noise_to_string(answer, AMOUNT_OF_NOISE)
        nn+=1
        if random_randint(10000) == 8: # Show some progress
            print (lolz)
            print(nn)
        pairs.append((question,answer))
    end = time.time()
    print(end - start) 
    tuple_pairs = tuple(pairs)
    pairs = None
    del pairs, answers
    gc.collect()
    return tuple_pairs

Uncomment and Run the following cell just once. this will do all the preprocessing and save preprocessed files in the drive. Hence the user won't have to carry out preprocessing every time he/she tries running the code.

In [0]:
#preprocesses_data_clean()
#preprocesses_data_analyze_chars()
#preprocesses_data_filter()
#preprocesses_split_lines()
#preprocess_partition_data()

In [0]:
pairs = generate_news_data()

In [0]:
#Run everything after this
PAD_token = 0  # Used for padding short sentences
SOS_token = 1  # Start-of-sentence token
EOS_token = 2  # End-of-sentence token
all_letters = string.ascii_letters + " .,);'(-"
alpha2index = {'☕': 0, "SOS":SOS_token, "EOS":EOS_token}
index2alpha = {PAD_token: "☕", SOS_token: "SOS", EOS_token: "EOS"}
for num,let in enumerate(all_letters):
    alpha2index[let] = num+3
    index2alpha[num+3] = let
num_words = len(alpha2index)

In [0]:
def binaryMatrix(L, value=alpha2index[PADDING]):
    m =[]
    for i,seq in enumerate(L):
        m.append([])
        for token in seq:
            if token==alpha2index[PADDING]:
                m[i].append(0)
            else:
                m[i].append(1)
    return m

def indexesFromSentence(sentence):
    return [alpha2index[alpha] for alpha in sentence] + [EOS_token]

def zeroPadding(l, fillvalue=PAD_token):
    return list(itertools.zip_longest(*l, fillvalue=fillvalue))

def inputVar(L):
    indexes_batch = [indexesFromSentence(sentt) for sentt in L]
    lengths = torch.tensor([len(index) for index in indexes_batch])
    padList = zeroPadding(indexes_batch)
    padVar = torch.LongTensor(padList)
    return padVar, lengths

def outputVar(L):
    indexes_batch = [indexesFromSentence(sentt) for sentt in L]
    max_target_len = max([len(index) for index in indexes_batch])
    padList = zeroPadding(indexes_batch)
    mask = binaryMatrix(padList)
    mask = torch.BoolTensor(mask)
    padVar = torch.LongTensor(padList)
    return padVar, mask, max_target_len
    
        
def batch2TrainData(pair_batch):
    pair_batch.sort(key=lambda x: len(x[0]), reverse=True)
    input_batch, output_batch = [],[]
    for pair in pair_batch:
        input_batch.append(pair[0])
        output_batch.append(pair[1])
    inp, lengths = inputVar(input_batch)
    output, mask, max_target_len = outputVar(output_batch)
    return inp, lengths, output, mask, max_target_len

small_batch_size = 3
batches = batch2TrainData([random.choice(pairs) for _ in range(small_batch_size)])
input_variable, lengths, target_variable, mask, max_target_len = batches

print("input_variable:", input_variable)
print("lengths:", lengths)
print("target_variable:", target_variable)
print("mask:", mask)
print("max_target_len:", max_target_len)

In [0]:
import torch.nn as nn
import torch.nn.functional as F
class EncoderRNN(nn.Module):
    def __init__(self, hidden_size, embedding, n_layers=1, dropout=0):
        super(EncoderRNN, self).__init__()
        self.n_layers = n_layers
        self.hidden_size = hidden_size
        self.embedding = embedding
        self.gru = nn.GRU(hidden_size, hidden_size, n_layers, dropout=dropout, bidirectional=True)
        
    def forward(self, input_seq, input_lengths, hidden=None):
        embedded = self.embedding(input_seq)
        packed = nn.utils.rnn.pack_padded_sequence(embedded, input_lengths)
        outputs,hidden = self.gru(packed, hidden)
        outputs,_ = nn.utils.rnn.pad_packed_sequence(outputs)
        outputs = outputs[:, :, :self.hidden_size] + outputs[:, : ,self.hidden_size:]
        return outputs, hidden
    

In [0]:
# Luong attention layer
class Attn(nn.Module):
    def __init__(self, method, hidden_size):
        super(Attn, self).__init__()
        self.method = method
        if self.method not in ['dot', 'general', 'concat']:
            raise ValueError(self.method, "is not an appropriate attention method.")
        self.hidden_size = hidden_size
        if self.method == 'general':
            self.attn = nn.Linear(self.hidden_size, hidden_size)
        elif self.method == 'concat':
            self.attn = nn.Linear(self.hidden_size * 2, hidden_size)
            self.v = nn.Parameter(torch.FloatTensor(hidden_size))

    def dot_score(self, hidden, encoder_output):
        return torch.sum(hidden * encoder_output, dim=2)

    def general_score(self, hidden, encoder_output):
        energy = self.attn(encoder_output)
        return torch.sum(hidden * energy, dim=2)

    def concat_score(self, hidden, encoder_output):
        energy = self.attn(torch.cat((hidden.expand(encoder_output.size(0), -1, -1), encoder_output), 2)).tanh()
        return torch.sum(self.v * energy, dim=2)

    def forward(self, hidden, encoder_outputs):
        # Calculate the attention weights (energies) based on the given method
        if self.method == 'general':
            attn_energies = self.general_score(hidden, encoder_outputs)
        elif self.method == 'concat':
            attn_energies = self.concat_score(hidden, encoder_outputs)
        elif self.method == 'dot':
            attn_energies = self.dot_score(hidden, encoder_outputs)

        # Transpose max_length and batch_size dimensions
        attn_energies = attn_energies.t()

        # Return the softmax normalized probability scores (with added dimension)
        return F.softmax(attn_energies, dim=1).unsqueeze(1)

In [0]:
class LuongAttnDecoderRNN(nn.Module):
    def __init__(self, attn_model, embedding, hidden_size, output_size, n_layers, dropout):
        super(LuongAttnDecoderRNN, self).__init__()
        
        self.attn_model = attn_model
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.n_layers = n_layers
        self.dropout=dropout
        
        self.embedding = embedding
        self.embedding_dropout = nn.Dropout(dropout)
        self.gru = nn.GRU(hidden_size, hidden_size, n_layers, dropout=dropout, bidirectional=False)
        self.concat = nn.Linear(hidden_size * 2, hidden_size)
        self.out = nn.Linear(self.hidden_size, self.output_size)
        self.attn = Attn(attn_model, hidden_size)
        
        
    def forward(self, input_seq, hidden, encoder_outputs):
        #input_seq = input_seq.view(1,-1)
        embedded = self.embedding(input_seq)
        embedded = self.embedding_dropout(embedded)
        rnn_output, hidden = self.gru(embedded, hidden)

        attn_weights = self.attn(rnn_output, encoder_outputs)
        context = attn_weights.bmm(encoder_outputs.transpose(0, 1))
        rnn_output = rnn_output.squeeze(0)
        context = context.squeeze(1)
        concat_input = torch.cat((rnn_output, context), 1)
        concat_output = torch.tanh(self.concat(concat_input))
        output = self.out(concat_output)
        output = F.softmax(output, dim=1)
        return output, hidden

In [0]:
def maskNLLLoss(inp, target, mask):
    nTotal = mask.sum()
    crossEntropy = -torch.log(torch.gather(inp, 1, target.view(-1, 1)).squeeze(1))
    loss = crossEntropy.masked_select(mask).mean()
    loss = loss.to(device)
    return loss, nTotal.item()

In [0]:
def train(input_variable, lengths, target_variable, mask, max_target_len, encoder, decoder, embedding, encoder_optimizer, 
          decoder_optimizer, batch_size, clip, max_length=CONFIG.max_input_len):
    
    encoder_optimizer.zero_grad()
    decoder_optimizer.zero_grad()
    
    #Setting device options
    input_variable = input_variable.to(device)
    target_variable = target_variable.to(device)
    lengths = lengths.to(device)
    mask = mask.to(device)
    
    #Initialize variables
    loss = 0
    print_losses = []
    n_totals = 0
    
    encoder_outputs, encoder_hidden = encoder(input_variable, lengths)
    #print("ENCODER Input - ", input_variable)

    decoder_input = torch.LongTensor([[SOS_token for _ in range(batch_size)]])
    decoder_input = decoder_input.to(device)
    
    decoder_hidden = encoder_hidden[:decoder.n_layers]
    
    use_teacher_forcing = True if random.random() < teacher_forcing_ratio else False
    temp_list = [] 

    nqueries=0                                #F
    total_fscore = 0                          #F
    total_accuracy = 0

    if use_teacher_forcing:
        for t in range(max_target_len):
            decoder_output, decoder_hidden = decoder(decoder_input, decoder_hidden, encoder_outputs)
            decoder_input = target_variable[t].view(1,-1)
            mask_loss, nTotal = maskNLLLoss(decoder_output, target_variable[t], mask[t])
            loss += mask_loss
            print_losses.append(mask_loss.item()*nTotal)
            n_totals += nTotal
            assert math.isnan(mask_loss)==False,print("mask Loss -",mask_loss, "decoder Output - ", decoder_output, "target_variable[t] -", target_variable[t])
            
            _, topi = decoder_output.topk(1) 
            y_true =  target_variable[t].cpu()
            y_pred = torch.squeeze(topi).cpu()
            fscore = f1_score(y_true, y_pred, average='micro')            
            total_fscore += fscore

            accuracy = accuracy_score(y_true, y_pred)
            total_accuracy += accuracy 
            nqueries +=1
            
    else:
        for t in range(max_target_len):
            decoder_output, decoder_hidden = decoder(decoder_input, decoder_hidden, encoder_outputs)#########Check
            _, topi = decoder_output.topk(1) 
            temp_list.append(topi)
            decoder_input = torch.LongTensor([[topi[i][0] for i in range(batch_size)]])
            decoder_input = decoder_input.to(device)
            mask_loss, nTotal = maskNLLLoss(decoder_output, target_variable[t], mask[t])
            loss += mask_loss
            print_losses.append(mask_loss.item()*nTotal)
            n_totals+=nTotal

            
            y_true =  target_variable[t].cpu()
            y_pred = torch.squeeze(topi).cpu()
            fscore = f1_score(y_true, y_pred, average='micro')        
            total_fscore += fscore

            accuracy = accuracy_score(y_true, y_pred)
            total_accuracy += accuracy 
            nqueries +=1

    F1_score = total_fscore/nqueries 
    acc_score = total_accuracy/nqueries 

    loss.backward()
    
    _ = nn.utils.clip_grad_norm_(encoder.parameters(), clip)
    _ = nn.utils.clip_grad_norm_(decoder.parameters(), clip)
    
    encoder_optimizer.step()
    decoder_optimizer.step()
    
    return sum(print_losses)/n_totals, F1_score,acc_score

In [0]:
def trainIters(model_name, pairs, encoder,decoder, encoder_optimizer, decoder_optimizer, embedding, encoder_n_layers,
               decoder_n_layers, save_dir, n_iteration, batch_size, print_every, save_every, clip, corpus_name, loadFilename):
    
    training_batches = [batch2TrainData([random.choice(pairs) for _ in range(batch_size)]) for _ in range(n_iteration)]
    print("initializing...")
    start_iteration = 1
    print_loss = 0

    net_f1score = 0                         #F
    f1batch_count = 0                         #F

    if loadFilename:
        start_iteration = checkpoint['iteration']+1
    print("Training...")
    for iteration in range(start_iteration,n_iteration+1):
        training_batch = training_batches[iteration-1]
        input_variable, lengths, target_variable, mask, max_target_len = training_batch
        loss, F1_score, acc_score = train(input_variable, lengths, target_variable, mask, max_target_len, encoder, decoder, 
                     embedding, encoder_optimizer, decoder_optimizer, batch_size, clip)
        print_loss+= loss

        if random.random()>0.9999:
          net_f1score = 0                         #F
          f1batch_count = 0 
          print_loss = 0
          print('---------------------------------------------------zeroing---------------------------------------------------')

        net_f1score += F1_score                         #F
        f1batch_count += 1                         #F
        F1_net = net_f1score/f1batch_count
        print_loss_avg = print_loss/f1batch_count

        if iteration % print_every == 0:
            print("Iteration : {};    % complete : {:0.1f}%;           Average Loss : {:.4f}%;            Avg Accuracy: {:.3f}%; Inst_accuracy : {:.3f}%".format(iteration, iteration/n_iteration*100, print_loss_avg*100, F1_net*100, acc_score*100))
            #print_loss=0
            
        if iteration % save_every == 0:
            directory = Path(save_dir + model_name + '/' + corpus_name + '/' + "{}-{}_{}".format(encoder_n_layers, decoder_n_layers, hidden_size))
            if not os.path.exists(directory):
                torch.save({
                    'iteration' : iteration,
                    'en' : encoder.state_dict(),
                    'de' : decoder.state_dict(),
                    'en_opt' : encoder_optimizer.state_dict(),
                    'de_opt' : decoder_optimizer.state_dict(),
                    'loss' : loss,
                    'embedding' : embedding.state_dict()
                },'/content/drive/My Drive/1-billion/deep_spell_attn/'+ '{}_{}.tar'.format(iteration, 'checkpoint'))

In [0]:
class GreedySearchDecoder(nn.Module):
    def __init__(self, encoder, decoder):
        super(GreedySearchDecoder, self).__init__()
        self.encoder = encoder
        self.decoder = decoder
        
    def forward(self, input_seq, input_length, max_length):
        encoder_outputs, encoder_hidden = self.encoder(input_seq, input_length)
        
        decoder_hidden = encoder_hidden[:decoder.n_layers]
        decoder_input = torch.ones(1, 1, device=device, dtype=torch.long)*SOS_token
        
        all_tokens = torch.zeros([0], device=device, dtype=torch.long)
        all_scores = torch.zeros([0], device=device)
        
        for _ in range(max_length):
            decoder_output, decoder_hidden = self.decoder(decoder_input, decoder_hidden, encoder_outputs)
            decoder_scores, decoder_input = torch.max(decoder_output, dim=1)
            
            all_tokens = torch.cat((all_tokens, decoder_input), dim=0)
            all_scores = torch.cat((all_scores, decoder_scores), dim=0)
            
            decoder_input = torch.unsqueeze(decoder_input, 0)
            
        return all_tokens, all_scores

In [0]:
def evaluate(encoder, decoder, searcher, sentence, max_length=CONFIG.max_input_len):
    ### Format input sentence as a batch
    # words -> indexes
    indexes_batch = [indexesFromSentence(sentence)]
    # Create lengths tensor
    lengths = torch.tensor([len(indexes) for indexes in indexes_batch])
    # Transpose dimensions of batch to match models' expectations
    input_batch = torch.LongTensor(indexes_batch).transpose(0, 1)
    # Use appropriate device
    input_batch = input_batch.to(device)
    lengths = lengths.to(device)
    # Decode sentence with searcher
    tokens, scores = searcher(input_batch, lengths, max_length)
    # indexes -> words
    decoded_words = [index2alpha[token.item()] for token in tokens]
    return decoded_words


def evaluateInput(encoder, decoder, searcher):
    input_sentence = ''
    while(1):
        try:
            # Get input sentence
            input_sentence = input('> ')
            # Check if it is quit case
            if input_sentence == 'q' or input_sentence == 'quit': break
            # Normalize sentence
            input_sentence = cleaned_text(input_sentence)
            # Evaluate sentence
            output_words = evaluate(encoder, decoder, searcher, input_sentence)
            # Format and print response sentence
            output_words[:] = [x for x in output_words if not (x == 'PAD')] #x == 'EOS' or 
            print('Bot:', ''.join(output_words))

        except KeyError:
            print("Error: Encountered unknown word.")

In [0]:
model_name = 'seq2seq_model'
attn_model = 'dot'
hidden_size = 500
encoder_n_layers = 2
decoder_n_layers = 2
dropout = 0.2
batch_size = 64

# Set checkpoint to load from; set to None if starting from scratch
loadFilename = "/content/drive/My Drive/1-billion/deep_spell_attn/240000_checkpoint.tar"
checkpoint_iter = 4000

if loadFilename:
    # If loading on same machine the model was trained on
    checkpoint = torch.load(loadFilename)
    # If loading a model trained on GPU to CPU
    #checkpoint = torch.load(loadFilename, map_location=torch.device('cpu'))
    encoder_sd = checkpoint['en']
    decoder_sd = checkpoint['de']
    encoder_optimizer_sd = checkpoint['en_opt']
    decoder_optimizer_sd = checkpoint['de_opt']
    embedding_sd = checkpoint['embedding']


print('Building encoder and decoder ...')
# Initialize word embeddings
embedding = nn.Embedding(num_words, hidden_size)
if loadFilename:
    embedding.load_state_dict(embedding_sd)
# Initialize encoder & decoder models
encoder = EncoderRNN(hidden_size, embedding, encoder_n_layers, dropout)
decoder = LuongAttnDecoderRNN(attn_model, embedding, hidden_size, num_words, decoder_n_layers, dropout)

if loadFilename:
    encoder.load_state_dict(encoder_sd)
    decoder.load_state_dict(decoder_sd)
# Use appropriate device
encoder = encoder.to(device)
decoder = decoder.to(device)
print('Models built and ready to go!')

Building encoder and decoder ...
Models built and ready to go!


In [1]:
'''save_dir = "/content/drive/My Drive/1-billion/deep_spell_attn/"
corpus_name="model"
# Configure training/optimization
clip = 50.0
teacher_forcing_ratio = 1
learning_rate = 0.0001
decoder_learning_ratio = 5.0
n_iteration = 300000
print_every = 10
save_every = 2500

# Ensure dropout layers are in train mode
encoder.train()
decoder.train()

# Initialize optimizers
print('Building optimizers ...')
encoder_optimizer = optim.Adam(encoder.parameters(), lr=learning_rate)
decoder_optimizer = optim.Adam(decoder.parameters(), lr=learning_rate * decoder_learning_ratio)
if loadFilename:
    encoder_optimizer.load_state_dict(encoder_optimizer_sd)
    decoder_optimizer.load_state_dict(decoder_optimizer_sd)

# If you have cuda, configure cuda to call
for state in encoder_optimizer.state.values():
    for k, v in state.items():
        if isinstance(v, torch.Tensor):
            state[k] = v.cuda()

for state in decoder_optimizer.state.values():
    for k, v in state.items():
        if isinstance(v, torch.Tensor):
            state[k] = v.cuda()

# Run training iterations
print("Starting Training!")
trainIters(model_name, pairs, encoder, decoder, encoder_optimizer, decoder_optimizer,
           embedding, encoder_n_layers, decoder_n_layers, save_dir, n_iteration, batch_size,
           print_every, save_every, clip, corpus_name, loadFilename)'''

'save_dir = "/content/drive/My Drive/1-billion/deep_spell_attn/"\ncorpus_name="model"\n# Configure training/optimization\nclip = 50.0\nteacher_forcing_ratio = 1\nlearning_rate = 0.0001\ndecoder_learning_ratio = 5.0\nn_iteration = 300000\nprint_every = 10\nsave_every = 2500\n\n# Ensure dropout layers are in train mode\nencoder.train()\ndecoder.train()\n\n# Initialize optimizers\nprint(\'Building optimizers ...\')\nencoder_optimizer = optim.Adam(encoder.parameters(), lr=learning_rate)\ndecoder_optimizer = optim.Adam(decoder.parameters(), lr=learning_rate * decoder_learning_ratio)\nif loadFilename:\n    encoder_optimizer.load_state_dict(encoder_optimizer_sd)\n    decoder_optimizer.load_state_dict(decoder_optimizer_sd)\n\n# If you have cuda, configure cuda to call\nfor state in encoder_optimizer.state.values():\n    for k, v in state.items():\n        if isinstance(v, torch.Tensor):\n            state[k] = v.cuda()\n\nfor state in decoder_optimizer.state.values():\n    for k, v in state.it

In [0]:
class GreedySearchDecoder(nn.Module):
    def __init__(self, encoder, decoder):
        super(GreedySearchDecoder, self).__init__()
        self.encoder = encoder
        self.decoder = decoder
        
    def forward(self, input_seq, input_length, max_length):
        encoder_outputs, encoder_hidden = self.encoder(input_seq, input_length)
        
        decoder_hidden = encoder_hidden[:decoder.n_layers]
        decoder_input = torch.ones(1, 1, device=device, dtype=torch.long)*SOS_token
        
        all_tokens = torch.zeros([0], device=device, dtype=torch.long)
        all_scores = torch.zeros([0], device=device)
        
        for _ in range(max_length):
            decoder_output, decoder_hidden = self.decoder(decoder_input, decoder_hidden, encoder_outputs)
            decoder_scores, decoder_input = torch.max(decoder_output, dim=1)
            
            all_tokens = torch.cat((all_tokens, decoder_input), dim=0)
            all_scores = torch.cat((all_scores, decoder_scores), dim=0)
            
            decoder_input = torch.unsqueeze(decoder_input, 0)
            
        return all_tokens, all_scores

In [0]:
def evaluate(encoder, decoder, searcher, sentence, max_length=CONFIG.max_input_len):
    ### Format input sentence as a batch
    # words -> indexes
    indexes_batch = [indexesFromSentence(sentence)]
    # Create lengths tensor
    lengths = torch.tensor([len(indexes) for indexes in indexes_batch])
    # Transpose dimensions of batch to match models' expectations
    input_batch = torch.LongTensor(indexes_batch).transpose(0, 1)
    # Use appropriate device
    input_batch = input_batch.to(device)
    lengths = lengths.to(device)
    # Decode sentence with searcher
    tokens, scores = searcher(input_batch, lengths, max_length)
    # indexes -> words
    decoded_words = [index2alpha[token.item()] for token in tokens]
    return decoded_words


def evaluateInput(encoder, decoder, searcher, pairr):
    # Get input sentence
    input_sentence = pairr[0]
    # Evaluate sentence
    output_words = evaluate(encoder, decoder, searcher, input_sentence)
    # Format and print response sentence
    output_words[:] = [x for x in output_words if not (x == 'PAD')] #
    predicted = ""
    for ip in output_words:
      if ip == "EOS":
        break
      else:
        predicted += ip
    #predicted = predicted.ljust(65)
    #target = pairr[1].ljust(65)
    target = [alpha2index[let] for let in pairr[1]]
    predicted = [alpha2index[let] for let in predicted]
    N=65
    target = (target + N * [''])[:N]
    predicted = (predicted + N * [''])[:N]
    fscore_eval = f1_score(target, predicted, average='micro')        

    accuracy_eval = accuracy_score(target, predicted)
    #print("Hello")
    return fscore_eval,accuracy_eval



Uncomment for Evaluation

In [0]:
encoder.eval()
decoder.eval()

tot_fscore_eval = 0
tot_accuracy_eval = 0

# Initialize search module
searcher = GreedySearchDecoder(encoder, decoder)

# Begin chatting (uncomment and run the following line to begin) 
for idx,pairr in enumerate(pairs):
  #print(idx)
  fscore_eval, accuracy_eval = evaluateInput(encoder, decoder, searcher, pairr)
  tot_fscore_eval += fscore_eval
  tot_accuracy_eval += accuracy_eval

  if (idx+1)%20 == 0:
    #print("________________________________")
    print("Iteration : {};     tot_fscore_eval is {:.3f}%       tot_accuracy_eval is {:.3f}%".format(idx+1, tot_fscore_eval*100/idx, tot_accuracy_eval*5))
    #tot_fscore_eval = 0
    tot_accuracy_eval = 0

